In [1]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

### Extract CSVs into DataFrames

In [2]:
IMDb_movies_file = "data/IMDb movies.csv"
IMDb_movies_df = pd.read_csv(IMDb_movies_file)
IMDb_movies_df.tail()

/Users/nimi/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
85850,tt9908390,Le lion,Le lion,2020,29/01/2020,Comedy,95,"France, Belgium",French,Ludovic Colbeau-Justin,...,"Dany Boon, Philippe Katerine, Anne Serra, Samu...",A psychiatric hospital patient pretends to be ...,5.3,398,NaN,NaN,"$3,507,171",NaN,NaN,4.0
85851,tt9911196,De Beentjes van Sint-Hildegard,De Beentjes van Sint-Hildegard,2020,13/02/2020,"Comedy, Drama",103,Netherlands,"German, Dutch",Johan Nijenhuis,...,"Herman Finkers, Johanna ter Steege, Leonie ter...",A middle-aged veterinary surgeon believes his ...,7.7,724,NaN,NaN,"$7,299,062",NaN,6.0,4.0
85852,tt9911774,Padmavyuhathile Abhimanyu,Padmavyuhathile Abhimanyu,2019,8/03/2019,Drama,130,India,Malayalam,Vineesh Aaradya,...,"Anoop Chandran, Indrans, Sona Nair, Simon Brit...",NaN,7.9,265,NaN,NaN,NaN,NaN,NaN,NaN
85853,tt9914286,Sokagin Çocuklari,Sokagin Çocuklari,2019,15/03/2019,"Drama, Family",98,Turkey,Turkish,Ahmet Faik Akinci,...,"Ahmet Faik Akinci, Belma Mamati, Metin Keçeci,...",NaN,6.4,194,NaN,NaN,"$2,833",NaN,NaN,NaN
85854,tt9914942,La vida sense la Sara Amat,La vida sense la Sara Amat,2019,5/02/2020,Drama,74,Spain,Catalan,Laura Jou,...,"Maria Morera Colomer, Biel Rossell Pelfort, Is...","Pep, a 13-year-old boy, is in love with a girl...",6.7,102,NaN,NaN,"$59,794",NaN,NaN,2.0


In [21]:
gg_awards_file = "data/golden_globe_awards.csv"
gg_awards_df = pd.read_csv(gg_awards_file)
gg_awards_df.head()

,year_film,year_award,ceremony,category,nominee,film,win
0,1943,1944,1,Best Performance by an Actress in a Supporting...,Katina Paxinou,For Whom The Bell Tolls,True
1,1943,1944,1,Best Performance by an Actor in a Supporting R...,Akim Tamiroff,For Whom The Bell Tolls,True
2,1943,1944,1,Best Director - Motion Picture,Henry King,The Song Of Bernadette,True
3,1943,1944,1,Picture,The Song Of Bernadette,NaN,True
4,1943,1944,1,Actress In A Leading Role,Jennifer Jones,The Song Of Bernadette,True


In [4]:
IMDb_ratings_file = "data/IMDb ratings.csv"
IMDb_ratings_df = pd.read_csv(IMDb_ratings_file)
IMDb_ratings_df.head()

,imdb_title_id,weighted_average_vote,total_votes,mean_vote,median_vote,votes_10,votes_9,votes_8,votes_7,votes_6,...,females_30age_avg_vote,females_30age_votes,females_45age_avg_vote,females_45age_votes,top1000_voters_rating,top1000_voters_votes,us_voters_rating,us_voters_votes,non_us_voters_rating,non_us_voters_votes
0,tt0000009,5.9,154,5.9,6.0,12,4,10,43,28,...,5.7,13.0,4.5,4.0,5.7,34.0,6.4,51.0,6.0,70.0
1,tt0000574,6.1,589,6.3,6.0,57,18,58,137,139,...,6.2,23.0,6.6,14.0,6.4,66.0,6.0,96.0,6.2,331.0
2,tt0001892,5.8,188,6.0,6.0,6,6,17,44,52,...,5.8,4.0,6.8,7.0,5.4,32.0,6.2,31.0,5.9,123.0
3,tt0002101,5.2,446,5.3,5.0,15,8,16,62,98,...,5.5,14.0,6.1,21.0,4.9,57.0,5.5,207.0,4.7,105.0
4,tt0002130,7.0,2237,6.9,7.0,210,225,436,641,344,...,7.3,82.0,7.4,77.0,6.9,139.0,7.0,488.0,7.0,1166.0


**Transform IMDb_movies_df**

In [23]:
# Create a filtered dataframe from specific columns
IMDb_movies_cols = ["imdb_title_id", "title","year","genre","country","avg_vote","worlwide_gross_income","reviews_from_users","reviews_from_critics"]
IMDb_movies_transformed= IMDb_movies_df[IMDb_movies_cols].copy()

# Rename the column headers
IMDb_movies_transformed = IMDb_movies_transformed.rename(columns={"imdb_title_id": "imdb_ID",
                                                              "title": "movie_title",
                                                              "year" : "year_released",
                                                              "genre" : "genre",
                                                              "country": "country",
                                                              "avg_vote": "avg_vote",
                                                              "worlwide_gross_income" : "worldwide_gross_income",
                                                              "reviews_from_users":"reviews_from_users",
                                                              "reviews_from_critics":"reviews_from_critics"
                                                         })

# Clean the data by dropping duplicates and setting the index
IMDb_movies_transformed.drop_duplicates("imdb_ID", inplace=True)
# IMDb_movies_transformed.set_index("id", inplace=True)


IMDb_movies_transformed.to_csv('IMDb_movies_transformed.csv')

IMDb_movies_transformed.head()

,imdb_ID,movie_title,year_released,genre,country,avg_vote,worldwide_gross_income,reviews_from_users,reviews_from_critics
0,tt0000009,Miss Jerry,1894,Romance,USA,5.9,NaN,1.0,2.0
1,tt0000574,The Story of the Kelly Gang,1906,"Biography, Crime, Drama",Australia,6.1,NaN,7.0,7.0
2,tt0001892,Den sorte drøm,1911,Drama,"Germany, Denmark",5.8,NaN,5.0,2.0
3,tt0002101,Cleopatra,1912,"Drama, History",USA,5.2,NaN,25.0,3.0
4,tt0002130,L'Inferno,1911,"Adventure, Drama, Fantasy",Italy,7.0,NaN,31.0,14.0


**Transform IMDb_ratings_df**

In [17]:
IMDb_ratings_transformed= IMDb_ratings_df.copy()

IMDb_ratings_transformed = pd.DataFrame({"imdb_ID":IMDb_ratings_df["imdb_title_id"],
                                   "imdb_rating":IMDb_ratings_df["mean_vote"],
                                   "total_votes":IMDb_ratings_df["total_votes"]      
                                       })
IMDb_ratings_tIMDb_ratings_transformed.to_csv('IMDb_ratings_transformed.csv')



**Transform golden_globe_awards dataframe**

In [18]:
gg_cols = ["year_film","year_award","category","nominee","film","win"]
Golden_globe_transformed = gg_awards_df[gg_cols].copy()

#Golden_globe_transformed = gg_awards_df.loc[gg_awards_df["win"] != False]

Golden_globe_transformed = pd.DataFrame({"movie_title":Golden_globe_transformed["film"],
                                         "nominee":Golden_globe_transformed["nominee"],
                                         "category": Golden_globe_transformed["category"],
                                        "year_released":Golden_globe_transformed["year_film"],
                                        "year_awarded":Golden_globe_transformed["year_award"],
                                        "win":Golden_globe_transformed["win"]
                                       })

Golden_globe_transformed.tail()

IMDb_ratings_transformed.head()
Golden_globe_transformed.to_csv('Golden_globe_transformed.csv')



**Create pandas visualisations for below:**
1. Film Nominations that won the most awards
2. Film Nominations that won atleast 5 awards
3. Film Nominations that were nominated more than 15 times
4. Film Nominations with highest imdb_ratings and wins>1
5. Film Nominations with highest imdb_ratings 
6. Film Nominations with highest total votes
7. Film Nominations with highest reviews_from_critics
8. Film Nominations with highest reviews_from_users

### Create database connection

In [11]:
connection_string = "postgres:tiki@localhost:5432/movies_db"
engine = create_engine(f'postgresql+psycopg2://{connection_string}')

In [12]:
# Confirm tables
engine.table_names()

<ipython-input-12-2d97e9123e98>:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['IMDB_movies', 'IMDB_ratings', 'Golden_globe']

### Load DataFrames into database

In [ ]:
IMDb_movies_transformed.to_sql(name='IMDB_movies', con=engine, if_exists='append', index=True)

In [ ]:
IMDB_ratings_transformed.to_sql(name='IMDB_ratings', con=engine, if_exists='append', index=True)

In [ ]:
Golden_globe_transformed.to_sql(name='Golden_globe', con=engine, if_exists='append', index=True)